<a href="https://colab.research.google.com/github/DJCordhose/ml-workshop/blob/master/notebooks/process/flask-server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serving using Flask

Links
* making flask server directly run on Colab: https://stackoverflow.com/questions/54465816/how-to-use-flask-in-google-colaboratory-python-notebook
* logging
  * https://flask.palletsprojects.com/en/1.1.x/logging/
  * https://docs.python.org/3/howto/logging-cookbook.html
* In a real production environment you would either use a "real" web server that directs to flask or use a hosted solution
  * https://flask.palletsprojects.com/en/1.1.x/deploying/
  * https://en.wikipedia.org/wiki/Web_Server_Gateway_Interface

In [10]:
!curl -O https://raw.githubusercontent.com/DJCordhose/ml-workshop/master/prod/prod_pipeline_pca_std_rf_2.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25098  100 25098    0     0  99595      0 --:--:-- --:--:-- --:--:-- 99595


In [0]:
!tar -xzf prod_pipeline_pca_std_rf_2.tgz

In [12]:
!ls -lR

.:
total 76
drwxr-xr-x 2 root root  4096 Aug  3 17:43 data
-rw-r--r-- 1 root root 32048 Aug  3 17:43 export-pipeline.ipynb
drwxr-xr-x 2 root root  4096 Aug  3 17:43 models
-rw-r--r-- 1 root root 25098 Aug  3 17:46 prod_pipeline_pca_std_rf_2.tgz
drwxr-xr-x 1 root root  4096 Aug  1 16:08 sample_data
drwxr-xr-x 2 root root  4096 Aug  3 17:43 stats

./data:
total 28
-rw-r--r-- 1 root root 26783 Aug  3 17:43 insurance-customers-1500.csv

./models:
total 60
-rw-r--r-- 1 root root 59322 Aug  3 17:43 model.pickle

./sample_data:
total 55504
-r-xr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Aug  1 16:08 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Aug  1 16:08 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Aug  1 16:08 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Aug  1 16:08 mnist_train_small.csv
-r-xr-xr-x 1 root root      930 Jan  1  2000 README.md

./stats:
total 12
-rw-r--r-- 1 root root 1145 Aug  3 17:43 describe.pick

In [0]:
import pickle

In [14]:
model_versions = pickle.load(open('stats/versions.pickle', 'rb'))
model_versions

{'model': 2,
 'model_name': 'pipeline_pca_std_rf',
 'numpy': '1.16.4',
 'pandas': '0.24.2',
 'sklearn': '0.21.3'}

In [15]:
import numpy, sklearn, pandas

versions = {
    'numpy': numpy.__version__,
    'sklearn': sklearn.__version__, 
    'pandas': pandas.__version__,
    'model': 2,
    'model_name': 'pipeline_pca_std_rf'
}
versions

{'model': 2,
 'model_name': 'pipeline_pca_std_rf',
 'numpy': '1.16.4',
 'pandas': '0.24.2',
 'sklearn': '0.21.3'}

In [0]:
assert model_versions == versions

In [0]:
import logging

logging.basicConfig(filename='req.log')

data_logger = logging.getLogger('DataLogger')
data_logger.setLevel(logging.INFO)

file_handler = logging.FileHandler('data.log', )

data_logger.addHandler(file_handler)

In [0]:
# loading models only once on startup
model = pickle.load(open('models/model.pickle', 'rb'))

def predict(speed, age, miles):
    sample = [[speed, age, miles]]

    result = int(model.predict(sample)[0])
    prediction = model.predict_proba(sample)[0].tolist()
    
    return result, prediction

In [19]:
# should be category 1
# * 0 - red: many accidents
# * 1 - blue: no accidents
# * 2 - yellow: few accidents   
predict(100, 48, 10)

(1, [0.010217113665389526, 0.6180901330470296, 0.371692753287581])

In [20]:
from flask import Flask, request, jsonify
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

app = Flask(__name__)

@app.route("/ping")
def ping():
    return "pong"

@app.route('/predict', methods=['GET', 'POST'])
def do_predict():
    speed = request.json['speed']
    age = request.json['age']
    miles = request.json['miles']

    predicted_category, probabilities = predict(speed, age, miles)

    response = {
        'category': predicted_category,
        'prediction': probabilities,
    }
    
    dataset = {
        'out': response,
        'in': {
            'speed': speed, 'age': age, 'miles': miles
        }
    }

    data_logger.info(dataset)

    return jsonify(response)
  
import threading
threading.Thread(target=app.run, kwargs={'host': '0.0.0.0', 'port': 80}).start()  

172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [21]:
import requests
r = requests.get("http://172.28.0.2/ping")
print(r.status_code)
print(r.encoding)
print(r.apparent_encoding)
print(r.text)

200
utf-8
ascii
pong


In [22]:
r = requests.post("http://172.28.0.2/predict", json={'speed': 100, 'age': 48, 'miles': 10})
r.status_code

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


200

In [23]:
response = r.json()
response

{'category': 1,
 'prediction': [0.010217113665389526, 0.6180901330470296, 0.37169275328758095]}

In [24]:
response['prediction']

[0.010217113665389526, 0.6180901330470296, 0.37169275328758095]

In [25]:
!cat data.log

{'out': {'category': 1, 'prediction': [0.010217113665389526, 0.6180901330470296, 0.37169275328758095]}, 'in': {'speed': 100, 'age': 48, 'miles': 10}}


# Exercise: deploy your model using this notebook

* load describe stats and scores as well and print them out
* how would you use them in a manual deployment process to find out if you want to deploy the new model at all?
* how would you automate the process?
* how could you use the logged out?